In [2]:
import whisper
import torch
from pydub import AudioSegment
import os
import base64
from io import BytesIO
import numpy

In [3]:
torch.cuda.empty_cache()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = whisper.load_model("medium")
model = model.to(device)

In [4]:
file_path = "/home/jongho/바탕화면/테스트 음성파일/"
file_name = "연제리2.wav"

encode_audio = base64.b64encode(open(f"{file_path}{file_name}", "rb").read()).decode('utf-8') 

In [66]:
import wave
import struct

def extract_wav_header(binary_data):
    # WAV 파일 헤더 구조에 맞게 필드를 읽음
  
    chunk_id = binary_data[:4].decode('utf-8')
    chunk_size = struct.unpack('<I', binary_data[4:8])[0]
    format_str = binary_data[8:12].decode('utf-8')
    subchunk1_id = binary_data[12:16].decode('utf-8')
    subchunk1_size = struct.unpack('<I', binary_data[16:20])[0]
    audio_format = struct.unpack('<H', binary_data[20:22])[0]
    num_channels = struct.unpack('<H', binary_data[22:24])[0]
    sample_rate = struct.unpack('<I', binary_data[24:28])[0]
    byte_rate = struct.unpack('<I', binary_data[28:32])[0]
    block_align = struct.unpack('<H', binary_data[32:34])[0]
    bits_per_sample = struct.unpack('<H', binary_data[34:36])[0]
    subchunk2_id = binary_data[36:40].decode('utf-8')
    subchunk2_size = struct.unpack('<I', binary_data[40:44])[0]
    

    header = {"Chunk ID:", chunk_id,
      "Chunk Size:", chunk_size,
      "Format:", format_str,
      "Subchunk1 ID:", subchunk1_id,
      "Subchunk1 Size:", subchunk1_size,
      "Audio Format:", audio_format,
      "Num Channels:", num_channels,
      "Sample Rate:", sample_rate,
      "Byte Rate:", byte_rate,
      "Block Align:", block_align,
      "Bits Per Sample:", bits_per_sample,
      "Subchunk2 ID:", subchunk2_id,
      "Subchunk2 Size:", subchunk2_size}
    # 헤더 정보 반환
    # header = {
    #     'channels': channels,
    #     'sample_width': sample_width,
    #     'frame_rate': frame_rate,
    #     'num_frames': num_frames
    # }
    
    return header 

In [67]:
# 웹 소켓을 통해서 base64 데이터를 수신 받았을때
# 바이너리 데이터로 변환
decoded_audio = base64.b64decode(encode_audio)

# 오디오 데이터에서 헤더정보 추출
BINARY_DATA = decoded_audio[:44]
audio_signal_data = decoded_audio[44:]

wav_header = extract_wav_header(BINARY_DATA)

print("음성파일 헤더 정보 :",wav_header)

음성파일 헤더 정보 : {1, 2, 'Block Align:', 4, 16, 176400, 'WAVE', 'Chunk ID:', 'RIFF', 'Sample Rate:', 'LIST', 543172920, 44100, 'Chunk Size:', 'Num Channels:', 80, 'fmt ', 'Subchunk2 Size:', 'Bits Per Sample:', 'Subchunk2 ID:', 'Format:', 'Subchunk1 ID:', 'Audio Format:', 'Byte Rate:', 'Subchunk1 Size:'}


In [61]:
# 분할 사이즈
SPLIT_SIZE = 10000000 # 10MB
# 기본 경로
BASE_PATH = "/home/jongho/Spring_project/CJU_STT/CJU_Web_Application_Services/src/main/webapp/resources/python/result"
# 분할 바이너리 데이터 리스트
split_data = []

for i in range(0, len(audio_signal_data), SPLIT_SIZE):
    split_data.append(audio_signal_data[i:i+SPLIT_SIZE])

print(f"바이너리 데이터 분할 완료 : {len(split_data)}개")

바이너리 데이터 분할 완료 : 55개


In [62]:
INPUT_DATA = split_data[20]

# WAV 파일 헤더 정보 생성
chunk_id = b'RIFF'
chunk_size = len(INPUT_DATA) + 36  # 데이터 크기에 추가 정보 크기(36바이트)를 더함
format_str = b'WAVE'
subchunk1_id = b'fmt '
subchunk1_size = 16  # 일반적인 값을 사용함
audio_format = 1  # PCM 포맷
num_channels = 2  # 스테레오
sample_rate = 44100  # 예시로 44100Hz
byte_rate = sample_rate * num_channels * 2  # 샘플링 레이트 * 채널 수 * 샘플 크기(2바이트)
block_align = num_channels * 2  # 채널 수 * 샘플 크기(2바이트)
bits_per_sample = 16  # 16비트 PCM 사용
subchunk2_id = b'data'
subchunk2_size = len(INPUT_DATA)  # 데이터 크기

wav_header = struct.pack('<4sI4s4sIHHIIHH4sI', chunk_id, chunk_size, format_str, subchunk1_id,
                         subchunk1_size, audio_format, num_channels, sample_rate, byte_rate,
                         block_align, bits_per_sample, subchunk2_id, subchunk2_size)

audio_with_header = wav_header + INPUT_DATA

with open('./audio_with_header.wav', 'wb') as file:
    file.write(audio_with_header)

In [63]:
# Whiper 입력 데이터 형식으로 변환
audio = AudioSegment.from_file(BytesIO(audio_with_header), format="wav")
data = numpy.array(audio.get_array_of_samples())
audio_float32 = data.astype('float32') / 32768.0

# STT 실행
result = model.transcribe('./audio_with_header.wav', fp16=False)

print(result['text'])

 이 체험을 과입해서 저희 보안에 있다 보니까 많은 분들이 작성하시면 없을 거예요. 그래서 일단은 제가 그거에 대한 안내는 따로 드릴 거고 지금 여기 드린 게 개인정보랑 보안에 걸려있다 보니까 제가 보안사약서랑 나중에 정보 찾기, 그리고 이제 뭐 주기가 쌓이도록 하거든요. 그래서 정보라든지 대상자 정보라든지 이런 것들이 있으니까 찾기 하면서, 이거를 나중에 제출해주시면 되고 부침 1이랑 부침 3은 제출을 해주시면 됩니다. 그 다음에 파일 보여드리기 위해서 스페어, 자폐정보가 필요하거든요. 그리고 보안교육은 개인정보공교류라고 해서 개인정보를 지금 다루기 때문에 그러니까 그거에 대한 그 기회가 우리 지금 들어가는 거예요.


In [6]:
# for audio_file_name in work_list :
#     save_path = "/home/jongho/Spring_project/CJU_STT/CJU_Web_Application_Services/src/main/webapp/resources/python/result"
#     target_audio_path = f"{save_path}/{audio_file_name}"
#     print(target_audio_path)
#     result = model.transcribe(target_audio_path, fp16=False)
#     print(result['text'])